In [1]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
# %pip install py_vollib
# %pip install pandas
# %pip install matplotlib

import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta
import py_vollib
from py_vollib.black_scholes.implied_volatility import implied_volatility
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 252 # T = time to expiration in years, 252 trading days
df = df.drop(["date"], axis=1)
df.head()

#interest rate data
df2 = pd.read_csv('../code/csv-data/daily-treasury-rates.csv')
df2 = df2[['Date', '13 WEEKS COUPON EQUIVALENT']]
df2['Date'] = pd.to_datetime(df2['Date'], format='%m/%d/%Y')
df2['Date'] = df2['Date'].dt.strftime('%Y-%m-%d')
df2 = df2[df2['Date'] >= '2023-08-21']
df2['13 WEEKS COUPON EQUIVALENT'] = df2['13 WEEKS COUPON EQUIVALENT'] / 100
df_result = pd.merge(df, df2, on='Date', how='left')
df_result.rename({"13 WEEKS COUPON EQUIVALENT": "Interest rate"}, axis="columns", inplace=True)
df_result['Interest rate'].fillna(df_result['Interest rate'].median(), inplace=True)
df_result.head()

,Date,Underlying,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190,T,Interest rate
0,2023-08-21,175.84,16.40,13.27,10.30,7.85,5.91,6.81,8.65,10.85,13.35,16.50,0.599206,0.0546
1,2023-08-22,177.23,17.50,14.10,11.10,8.51,6.34,6.26,8.10,10.10,12.75,15.50,0.595238,0.0546
2,2023-08-23,181.12,20.20,16.65,13.45,10.55,8.08,5.25,6.71,8.50,10.70,13.33,0.591270,0.0546
3,2023-08-24,176.38,16.97,13.75,10.80,8.35,6.25,6.85,8.62,10.75,13.35,16.60,0.587302,0.0547
4,2023-08-25,178.61,18.50,15.15,11.85,9.19,6.90,5.90,7.50,9.50,11.90,14.81,0.583333,0.0550


# Delta-vega hedging a single option

In [2]:
def get_greeks_for_option(option, call_vola, put_vola, strike):
    call_bsm = BSM(kind='call', S0=option['Underlying'], K=strike, T=option['T'], r=option['Interest rate'], sigma=call_vola)
    put_bsm = BSM(kind='put', S0=option['Underlying'], K=strike, T=option['T'], r=option['Interest rate'], sigma=put_vola)
    return call_bsm.delta(), call_bsm.gamma(), call_bsm.vega(), put_bsm.delta(), put_bsm.gamma(), put_bsm.vega()
    

In [15]:
# TODO: refactor when time
def delta_vega_hedge(option_data, freq):
    # Initial values at time t = 0
    initial = option_data.iloc[0]
    option_amount = 1000
    interest_rate = initial['Interest rate']
    stock_price = initial['Underlying']
    call_option_price = initial['C170']
    put_option_price = initial['P170']
    strike = 170
    time_to_maturity = initial['T']
    total_mean_squared_error = []
    
    call_volatility = implied_volatility(price=call_option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='c')
    put_volatility = implied_volatility(price=put_option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='p')

    # Initial stuff at t = 0
    call_bsm = BSM(kind='call', S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=call_volatility)
    put_bsm = BSM(kind='put', S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=put_volatility)    

    # REMEMBER: Delta of the underlying stock is 1, Gamma is 0, Vega is 0
    # To make the portfolio Gamma and Vega neutral, we have to take both call and put options
    # Gamma expresses how much hedging will cost in a small time interval
    # If Gamma of the portfolio is positive, we make money by delta hedging and vice versa
    # With both Delta and Gamma hedging, we obtain a portfolio, which is even less sensitive to small price changes of the underlying
    # Need to rebalance the hedge for a Delta and Gamma neutral portfolio occurs less often, which saves trading costs
    # Delta neutrality can be achieved by fixing the option positions
    
    call_delta = call_bsm.delta()
    call_gamma = call_bsm.gamma()
    call_vega = call_bsm.vega()
    
    put_delta = put_bsm.delta()
    put_gamma = put_bsm.gamma()
    put_vega = put_bsm.vega()
    
    print('Call delta', call_delta)
    print('Call gamma', call_gamma)
    print('Call vega', call_vega)
    
    print('Put delta', put_delta)
    print('Put delta', put_gamma)
    print('Put delta', put_vega)

    portfolio_call_delta = option_amount * call_delta
    portfolio_call_gamma = option_amount * call_gamma
    portfolio_call_vega = option_amount * call_vega
    
    print('Portfolio Call delta', portfolio_call_delta)
    print('portfolio Call gamma', portfolio_call_gamma)
    print('Portfolio Call vega', portfolio_call_vega)

    portfolio_put_delta = option_amount * put_delta
    portfolio_put_gamma = option_amount * put_gamma
    portfolio_put_vega = option_amount * put_vega

    print('Portfolio Put delta', portfolio_put_delta)
    print('Portfolio Put gamma', portfolio_put_gamma)
    print('Portfolio Put vega', portfolio_put_vega)

    # Delta and Vega for put and call options
    delta_vega_greeks = [[call_delta, call_vega], [put_delta, put_vega]]

    # Delta and Gamma for put and call options
    delta_gamma_greeks = [[call_delta, call_gamma], [put_delta, put_gamma]]
    
    # Delta and Vega for initial portfolios
    portfolios_delta_vega = [portfolio_call_delta + portfolio_put_delta, portfolio_call_vega + portfolio_put_vega]

    # Delta and Gamma for initial portfolios
    portfolios_delta_gamma = [portfolio_call_delta + portfolio_put_delta, portfolio_call_gamma + portfolio_put_gamma]
    
    portfolio_value_calls = option_amount * call_option_price
    print(portfolio_value_calls)
    
    portfolio_value_puts = option_amount * put_option_price
    print(portfolio_value_puts)
    
    # Solve when the linear equation is zero
    # This makes the portfolio delta and vega neutral
    delta_vega_neutral = np.linalg.solve(delta_vega_greeks, portfolios_delta_vega)
    print(delta_vega_neutral)

    # Solve when the linear equation is zero
    # This makes the portfolio Delta and Gamma neutral
    delta_gamma_neutral = np.linalg.solve(delta_gamma_greeks, portfolios_delta_gamma)
    print(delta_gamma_neutral)
    
    # Loop through dates in the dataframe
    for i in range(1, len(option_data)-1):
        pass
        # Rehedge interval
        # Compute new Deltas, Vegas, and Gammas every time we want to rehedge
        # We need to rehedge, because as time goes by, the price of the underlying changes, which affects the option Greeks that measure the market
        if i % freq == 0:
            call_volatility = implied_volatility(price=option_data['C170'][i], S=option_data['Underlying'][i], K=170, t=option_data['T'][i], r=option_data['Interest rate'][i], flag='c')
            put_volatility = implied_volatility(price=option_data['P170'][i], S=option_data['Underlying'][i], K=170, t=option_data['T'][i], r=option_data['Interest rate'][i], flag='p')
            call_delta, call_gamma, call_vega, put_delta, put_gamma, put_vega = get_greeks_for_option(option_data.iloc[i], call_volatility, put_volatility, strike)
            
            delta_gamma_neutral = np.linalg.solve([[call_delta, call_gamma], [put_delta, put_gamma]], [option_amount * (call_delta + put_delta), option_amount * (call_gamma + put_gamma)])
            print('To make the portfolio delta gamma neutral, you need to adjust the amount of call options: ', delta_gamma_neutral[0])
            print('To make the portfolio delta gamma neutral, you need to adjust the amount of put options: ', delta_gamma_neutral[1])
            
            delta_vega_neutral = np.linalg.solve([[call_delta, call_vega,], [put_delta, put_vega]], [option_amount * (call_delta + put_delta), option_amount * (call_vega + put_vega)])
            print('To make the portfolio delta gamma neutral, you need to adjust the amount of call options: ', delta_vega_neutral[0])
            print('To make the portfolio delta gamma neutral, you need to adjust the amount of put options: ', delta_vega_neutral[1])

            coefficients = np.array([
                [call_delta, call_gamma, call_vega],
                [put_delta, put_gamma, put_vega],
                [1, 0, 0]
            ])

            right_hand_side = np.array([
                option_amount * (call_delta + put_delta),
                option_amount * (call_gamma + put_gamma),
                option_amount * (call_vega + put_vega)
            ])
            
            delta_vega_gamma_neutral = np.linalg.solve(coefficients, right_hand_side)
            if delta_vega_gamma_neutral[0] < 0:
                print("Short this many units of the underlying", delta_vega_gamma_neutral[0])
            else:
                print("Buy this many units of the underlying", delta_vega_gamma_neutral[0])
            print("To adjust portfolio to be delta-vega-gamma neutral, you need to balance your calls and puts the following way: ")
            print("Adjust calls: ", delta_vega_gamma_neutral[1])
            print("Adjust puts: ", delta_vega_gamma_neutral[2])


In [16]:
delta_vega_hedge(df_result, freq=1)

Call delta 0.7039443038937685
Call gamma 0.013719648918889866
Call vega 47.041577446648766
Put delta -0.31769685085392885
Put delta 0.011849366066525684
Put delta 48.52857130915117
Portfolio Call delta 703.9443038937685
portfolio Call gamma 13.719648918889867
Portfolio Call vega 47041.577446648764
Portfolio Put delta -317.6968508539288
Portfolio Put gamma 11.849366066525684
Portfolio Put vega 48528.57130915117
16400.0
6810.0
[-91169.97044914   1372.50560762]
[  332.75552836 11079.4449013 ]
To make the portfolio delta gamma neutral, you need to adjust the amount of call options:  370.45294053115083
To make the portfolio delta gamma neutral, you need to adjust the amount of put options:  11702.123436275722
To make the portfolio delta gamma neutral, you need to adjust the amount of call options:  -89235.8144870668
To make the portfolio delta gamma neutral, you need to adjust the amount of put options:  1404.3798493719648
Buy this many units of the underlying 93608.05849275902
To adjust po